### I'm not doing this for now because it's prob not a big issue

but leaving the code here in case i want to come back to it...

```
df_f_mentions["entities.mentions.usernames"] = df_f_mentions["entities.mentions"].apply(extract_usernames)
df_f_mentions["entities.mentions.num_mentions"] = df_f_mentions["entities.mentions"].apply(extract_num_mentions)
df_f_mentions["entities.mentions.double_mention"] = df_f_mentions["entities.mentions"].apply(extract_double_mention)
df_f_mentions["tweet_type"] = df_f_mentions.apply(lambda x: extract_tweet_type(x), axis=1)

df_f_mentions["num_mentions_received"] = df_f_mentions.groupby("entities.mentions.usernames")["id"].transform("count")
```

In [1]:
import os
import datetime
from datetime import timedelta
import pandas as pd
import numpy as np
from wordcloud import WordCloud
import matplotlib.pyplot as plt
import plotly.express as px

import twarc
from twarc import Twarc2, expansions
from twarc_csv import DataFrameConverter


from dotenv import load_dotenv

load_dotenv()
from data import load_data, get_time_interval, save_user_following, extract_tweet_type

In [2]:
USER = "nicktorba"
HOURS = 24

end_time, start_time = get_time_interval(HOURS)
print(end_time, "    ", start_time)

2022-06-23T14:03:01Z      2022-06-22T14:03:01Z


In [3]:
def get_user_following_tweets(client, following_ids: list, fpath: str, start_time: str, end_time: str):
    """
    
    """
    df_tweets = None
    df_ref_tweets = None 
    
    converter = DataFrameConverter("tweets", allow_duplicates=True) # allow duplicates because I'm process data and includes separate, so it wasn't allowing all tweets when going through the includes
    for id_ in following_ids:
        timeline_gen = client.timeline(id_, start_time=start_time, end_time=end_time)
        
        for res in timeline_gen:
            df_tweets_next = converter.process([res])
            
            if df_tweets is None:
                df_tweets = df_tweets_next
            else: 
                df_tweets = pd.concat([df_tweets, df_tweets_next])
                
            if "tweets" in res["includes"]:
                df_ref_tweets_next = converter.process(res["includes"]["tweets"])
                if df_ref_tweets is None:
                    df_ref_tweets = df_ref_tweets_next
                else: 
                    df_ref_tweets = pd.concat([df_ref_tweets, df_ref_tweets_next])
            
    return df_tweets, df_ref_tweets


def get_user_following_mentions(client: twarc.client2.Twarc2, following_ids: list, start_time: str, end_time: str):
    """
    Get all tweets that mention the accounts included in following_ids 
    
    This func is limited by the search api, which is only the past 7 days
    """

    converter = DataFrameConverter("tweets", inline_referenced_tweets=False) #includes "included" tweets, aka referenced tweets
    df = None
    for username in following_ids:
        query = f"to:{username}"
        for i in client.search_recent(query, start_time=start_time, end_time=end_time):
            if df is None:
                df = converter.process([i])
            else: 
                df = pd.concat([df, converter.process([i])])
    return df


In [4]:
client = Twarc2(
    consumer_key=os.environ["consumer_key"], 
    consumer_secret=os.environ["consumer_secret"],
    access_token=os.environ["access_token"], 
    access_token_secret=os.environ["access_token_secret"]
)

df_following = save_user_following(client, USER, f"data/nicktorba/following-{end_time}.csv")

get_tweets_from = df_following["id"].tolist()[:5]
get_tweets_from.append(881192533056827392)

df_tweets, df_ref_tweets = get_user_following_tweets(client, get_tweets_from, f"data/nicktorba/TEST-following-tweets-{end_time}.csv", start_time, end_time)

df_tweets["tweet_type"] = df_tweets.apply(lambda x: extract_tweet_type(x), axis=1)
df_ref_tweets["tweet_type"] = df_ref_tweets.apply(lambda x: extract_tweet_type(x), axis=1)

In [6]:
df_tweets["tweet_link"] = df_tweets.apply(lambda row: f"https://twitter.com/{row['author.username']}/status/{row.id}", axis=1)
df_ref_tweets["tweet_link"] = df_ref_tweets.apply(lambda row: f"https://twitter.com/{row['author.username']}/status/{row.id}", axis=1)

In [ ]:
df_tweets["author.username"].unique()

In [ ]:
df_tweets["tweet_type"].unique()

In [7]:
df_user = df_tweets[(df_tweets["author.username"] == "goblinodds") ]

df_user_mentions = get_user_following_mentions(client, ["goblinodds"], start_time, end_time)
df_user_mentions["tweet_link"] = df_user_mentions.apply(lambda row: f"https://twitter.com/{row['author.username']}/status/{row.id}", axis=1)

df_user_mentions["tweet_type"] = df_user_mentions.apply(lambda x: extract_tweet_type(x), axis=1)


df_user["tweet_type"].value_counts()

reply         27
self-reply     2
standalone     2
rt             1
Name: tweet_type, dtype: int64

In [8]:
df_user["public_metrics.reply_count"].value_counts()

0     16
1     14
3      1
12     1
Name: public_metrics.reply_count, dtype: int64

In [9]:
df_user[df_user["public_metrics.reply_count"]==12].id

20    1539727872931180544
Name: id, dtype: object

In [11]:
df_user_mentions[df_user_mentions["referenced_tweets.replied_to.id"] == "1539727872931180544"].shape

(12, 75)

In [24]:
df_user_mentions[df_user_mentions["referenced_tweets.replied_to.id"] == "1539793549830545411"]["author.username"]

df_user_mentions[df_user_mentions["referenced_tweets.replied_to.id"] == "1539727872931180544"]["author.username"]

0           YogaFanGG
13           JonBaszk
15     sympatheticopp
20    panchromaticity
21          tomabella
24           HDPbilly
25    arachnocapital2
27         goblinodds
28       SarahFPoetry
29            palecur
30    chopstickfury01
31    wordsnearmyname
Name: author.username, dtype: object

In [33]:
df_user_mentions["tweet_type"] = df_user_mentions.apply(lambda x: extract_tweet_type(x), axis=1)

df_user_mentions[df_user_mentions["author.username"] == "goblinodds"].loc[26].tweet_link

'https://twitter.com/goblinodds/status/1539729495883583488'

In [35]:
df_user_mentions[df_user_mentions["author.username"] == "goblinodds"].to_csv("tests/self-reply.csv")

In [30]:
df_user_mentions[df_user_mentions["tweet_type"]=="reply"]

,id,conversation_id,referenced_tweets.replied_to.id,referenced_tweets.retweeted.id,referenced_tweets.quoted.id,author_id,in_reply_to_user_id,retweeted_user_id,quoted_user_id,created_at,...,geo.id,geo.name,geo.place_id,geo.place_type,__twarc.retrieved_at,__twarc.url,__twarc.version,,tweet_link,tweet_type
0,1539877925159313409,1539727872931180544,1539727872931180544,NaN,NaN,1436059240364449794,881192533056827392,NaN,NaN,2022-06-23T07:48:01.000Z,...,NaN,NaN,NaN,NaN,2022-06-23T14:03:20+00:00,https://api.twitter.com/2/tweets/search/recent...,2.10.4,NaN,https://twitter.com/YogaFanGG/status/153987792...,reply
1,1539855923853725696,1539793549830545411,1539793551172571136,NaN,NaN,71927679,881192533056827392,NaN,NaN,2022-06-23T06:20:36.000Z,...,NaN,NaN,NaN,NaN,2022-06-23T14:03:20+00:00,https://api.twitter.com/2/tweets/search/recent...,2.10.4,NaN,https://twitter.com/Be_More_Weird/status/15398...,reply
2,1539842365887750144,1537476600643694593,1537476600643694593,NaN,NaN,122732341,881192533056827392,NaN,NaN,2022-06-23T05:26:43.000Z,...,NaN,NaN,NaN,NaN,2022-06-23T14:03:20+00:00,https://api.twitter.com/2/tweets/search/recent...,2.10.4,NaN,https://twitter.com/Idonbelonghere/status/1539...,reply
3,1539839084373282818,1507035972822654980,1507035972822654980,NaN,NaN,1528248717693685761,881192533056827392,NaN,NaN,2022-06-23T05:13:41.000Z,...,NaN,NaN,NaN,NaN,2022-06-23T14:03:20+00:00,https://api.twitter.com/2/tweets/search/recent...,2.10.4,NaN,https://twitter.com/beginanon/status/153983908...,reply
4,1539805191641464834,1507035972822654980,1539802974100127745,NaN,NaN,1716999434,881192533056827392,NaN,NaN,2022-06-23T02:59:00.000Z,...,NaN,NaN,NaN,NaN,2022-06-23T14:03:20+00:00,https://api.twitter.com/2/tweets/search/recent...,2.10.4,NaN,https://twitter.com/adwaits16/status/153980519...,reply
5,1539803298898878464,1242949961743925248,1539803037182468096,NaN,NaN,1163927704049262592,881192533056827392,NaN,NaN,2022-06-23T02:51:29.000Z,...,NaN,NaN,NaN,NaN,2022-06-23T14:03:20+00:00,https://api.twitter.com/2/tweets/search/recent...,2.10.4,NaN,https://twitter.com/strangestloop/status/15398...,reply
6,1539796316124897281,1539793549830545411,1539793549830545411,NaN,NaN,901960840215052288,881192533056827392,NaN,NaN,2022-06-23T02:23:44.000Z,...,NaN,NaN,NaN,NaN,2022-06-23T14:03:20+00:00,https://api.twitter.com/2/tweets/search/recent...,2.10.4,NaN,https://twitter.com/panchromaticity/status/153...,reply
7,1539795708047331328,1539793549830545411,1539793549830545411,NaN,NaN,1244661469699956737,881192533056827392,NaN,NaN,2022-06-23T02:21:19.000Z,...,NaN,NaN,NaN,NaN,2022-06-23T14:03:20+00:00,https://api.twitter.com/2/tweets/search/recent...,2.10.4,NaN,https://twitter.com/acidshill/status/153979570...,reply
9,1539770550272380928,1538683917334548482,1538683917334548482,NaN,NaN,1331763711908995074,881192533056827392,NaN,NaN,2022-06-23T00:41:21.000Z,...,NaN,NaN,NaN,NaN,2022-06-23T14:03:20+00:00,https://api.twitter.com/2/tweets/search/recent...,2.10.4,NaN,https://twitter.com/karma_gardener/status/1539...,reply
10,1539767282939068416,1242949961743925248,1242949961743925248,NaN,NaN,864063973938741250,881192533056827392,NaN,NaN,2022-06-23T00:28:22.000Z,...,NaN,NaN,NaN,NaN,2022-06-23T14:03:20+00:00,https://api.twitter.com/2/tweets/search/recent...,2.10.4,NaN,https://twitter.com/kureshii_/status/153976728...,reply


In [22]:
df_user[df_user["id"] == "1539793549830545411"].iloc[0].tweet_link

'https://twitter.com/goblinodds/status/1539793549830545411'

In [14]:
df_user[df_user["tweet_type"]=="self-reply"]

,id,conversation_id,referenced_tweets.replied_to.id,referenced_tweets.retweeted.id,referenced_tweets.quoted.id,author_id,in_reply_to_user_id,retweeted_user_id,quoted_user_id,created_at,...,geo.id,geo.name,geo.place_id,geo.place_type,__twarc.retrieved_at,__twarc.url,__twarc.version,,tweet_type,tweet_link
4,1539793551172571136,1539793549830545411,1539793549830545411,NaN,NaN,881192533056827392,881192533056827392,NaN,NaN,2022-06-23T02:12:45.000Z,...,NaN,NaN,NaN,NaN,2022-06-23T14:03:10+00:00,https://api.twitter.com/2/users/88119253305682...,2.10.4,NaN,self-reply,https://twitter.com/goblinodds/status/15397935...
19,1539729295571943424,1539727872931180544,1539727872931180544,NaN,NaN,881192533056827392,881192533056827392,NaN,NaN,2022-06-22T21:57:25.000Z,...,NaN,NaN,NaN,NaN,2022-06-23T14:03:10+00:00,https://api.twitter.com/2/users/88119253305682...,2.10.4,NaN,self-reply,https://twitter.com/goblinodds/status/15397292...


In [ ]:
df_user["entities.mentions"]

In [ ]:
df_user_mentions.columns

In [ ]:
df_user["public_metrics.reply_count"].value_counts()

In [ ]:
df_user[df_user["public_metrics.reply_count"] > 0].iloc[1].tweet_link

In [ ]:
for i in df_user_mentions["tweet_link"].tolist():
    print(i)

In [ ]:
df_tweets["author.id"].unique()

In [ ]:
for i in df_tweets[df_tweets["referenced_tweets.replied_to.id"].notna()]["referenced_tweets.replied_to.id"].tolist():
    print(i in df_ref_tweets["id"].tolist())

In [ ]:
df_ref_tweets["id"].value_counts()

In [ ]:
print(df_tweets[df_tweets["referenced_tweets.replied_to.id"].notna()].iloc[3]["referenced_tweets.replied_to.id"])

df_tweets[df_tweets["referenced_tweets.replied_to.id"].notna()].iloc[3].tweet_link

In [ ]:
'1539606569708560385' in df_ref_tweets["id"]

In [ ]:
df_ref_tweets["created_at"].max()

In [ ]:
df_ref_tweets[df_ref_tweets["id"] == '1539252786554146816']

In [ ]:
df_ref_tweets["author.username"]